In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 2    # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_exact_50-2.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [3:17:49<00:00, 11.87s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      2.644320               0.0     11.869477
std       3.389771               0.0     12.450821
min       0.000435               0.0      0.407894
25%       0.185123               0.0      3.910013
50%       1.272357               0.0      7.738960
75%       3.942360               0.0     14.801863
max      20.850715               0.0     60.581608
Number of infeasible solution: 0


## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rnd_50-2.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:30:36<00:00,  9.04s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     11.149628          0.111986      9.034461
std       6.088041          0.652292     16.096517
min       0.097133          0.000000      0.667735
25%       6.480240          0.000000      0.901159
50%      10.735721          0.000000      1.026443
75%      14.712260          0.000000      8.286686
max      32.457415          8.221280     60.685417
Number of infeasible solution: 94


## Heuristic - RENS

In [9]:
from src.heuristic import rens

In [10]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = rens(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rens_50-2.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [3:01:31<00:00, 10.89s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     12.955519          0.018483     10.889503
std     112.172757          0.232786     16.527420
min       0.000434          0.000000      1.349094
25%       0.218668          0.000000      2.044667
50%       1.374716          0.000000      2.622241
75%       4.360096          0.000000     11.044856
max    2243.511953          4.992248     62.713541
Number of infeasible solution: 11


## Heuristic - N1

In [7]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-2.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:17<00:00, 12.88it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean    882.322766               0.0      0.077087
std     606.229289               0.0      0.031392
min     123.742255               0.0      0.058603
25%     304.856171               0.0      0.061765
50%     779.677821               0.0      0.062896
75%    1478.662998               0.0      0.077385
max    2021.366242               0.0      0.233990
Number of infeasible solution: 0


## Heuristic - N3

In [9]:
model_heur = model.first_solution_heuristic(nodes_limit=3)

In [10]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n3_50-2.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:38<00:00,  2.96it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean    201.699859               0.0      0.337325
std     395.526371               0.0      2.338168
min       0.005218               0.0      0.059640
25%      11.565499               0.0      0.077842
50%      35.023988               0.0      0.109851
75%     134.055706               0.0      0.141296
max    1608.123487               0.0     58.055639
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [8]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd])
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [9]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup)
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 303.71
Epoch 1, Validation Loss: 60.11
Epoch 2, Validation Loss: 47.18
Epoch 3, Validation Loss: 27.07
Epoch 4, Validation Loss: 29.75
Epoch 5, Validation Loss: 26.36
Epoch 6, Validation Loss: 29.61
Epoch 7, Validation Loss: 24.70
Epoch 8, Validation Loss: 21.45
Epoch 9, Validation Loss: 18.96
Epoch 10, Validation Loss: 18.91
Epoch 11, Validation Loss: 16.38
Epoch 12, Validation Loss: 15.42
Epoch 13, Validation Loss: 15.13
Epoch 14, Validation Loss: 16.00
Epoch 15, Validation Loss: 14.66
Epoch 16, Validation Loss: 13.11
Epoch 17, Validation Loss: 13.87
Epoch 18, Validation Loss: 12.45
Epoch 19, Validation Loss: 12.49
Epoch 20, Validation Loss: 12.89
Epoch 21, Validation Loss: 11.60
Epoch 22, Validation Loss: 12.32
Epoch 23, Validation Loss: 18.30
Epoch 24, Validation Loss: 12.62
Epoch 25, Validation Loss: 14.17
Epoch 26, Validation Loss: 11.86
Epoch 27, Validation Loss: 12.30
Epoch 28, Validation Loss: 13.54
Epoch 29, Validation Loss: 12.34
Epoch 30, Validatio

In [10]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lr_50-2.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 794.27it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      8.532119               0.0      0.000771
std       7.696579               0.0      0.000533
min       0.048154               0.0      0.000000
25%       2.947259               0.0      0.000000
50%       6.088938               0.0      0.001000
75%      11.753885               0.0      0.001000
max      48.586977               0.0      0.002522
Number of infeasible solution: 0


## Learnable Threshold

In [11]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [12]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd])
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [13]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup)
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 238.91
Epoch 1, Validation Loss: 34.20
Epoch 2, Validation Loss: 17.75
Epoch 3, Validation Loss: 20.35
Epoch 4, Validation Loss: 18.31
Epoch 5, Validation Loss: 15.81
Epoch 6, Validation Loss: 12.73
Epoch 7, Validation Loss: 18.15
Epoch 8, Validation Loss: 14.26
Epoch 9, Validation Loss: 13.26
Epoch 10, Validation Loss: 15.04
Epoch 11, Validation Loss: 13.86
Epoch 12, Validation Loss: 13.40
Epoch 13, Validation Loss: 12.56
Epoch 14, Validation Loss: 15.41
Epoch 15, Validation Loss: 12.95
Epoch 16, Validation Loss: 13.30
Epoch 17, Validation Loss: 14.26
Epoch 18, Validation Loss: 12.86
Epoch 19, Validation Loss: 12.58
Epoch 20, Validation Loss: 12.59
Epoch 21, Validation Loss: 12.52
Epoch 22, Validation Loss: 12.64
Epoch 23, Validation Loss: 17.13
Epoch 24, Validation Loss: 12.58
Epoch 25, Validation Loss: 12.49
Epoch 26, Validation Loss: 13.01
Epoch 27, Validation Loss: 14.26
Epoch 28, Validation Loss: 13.51
Epoch 29, Validation Loss: 13.76
Epoch 30, Validatio

In [14]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lt_50-2.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 686.30it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     11.760018          0.000318      0.000893
std       8.256438          0.003896      0.000603
min       0.025281          0.000000      0.000000
25%       4.992462          0.000000      0.000504
50%       9.755346          0.000000      0.001000
75%      17.566854          0.000000      0.001006
max      45.645138          0.075537      0.003232
Number of infeasible solution: 9
